In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor
import numpy as np
import autogluon.core as ag
filename = input("Filename: ")
train_data = pd.read_csv('filename')
a = 1
train_data['item_id'] = 0
for i in range(0,len(train_data),10*24):
    train_data['item_id'][i:i+10*24] = a*np.ones((10*24))
    a = a+1

In [ ]:
train_data_partial_per_item = []
for item_id in train_data.item_ids:
    timestamps = train_data.loc[item_id][:48].index.get_level_values("timestamp")
    index = pd.MultiIndex.from_product([[item_id], timestamps], names=["item_id", "timestamp"])
    train_data_partial_per_item.append(pd.DataFrame(train_data.loc[item_id][:48].values, index = index, columns = ["Temp_Box", "Temp"]))
train_data_partial = TimeSeriesDataFrame(pd.concat(train_data_partial_per_item))

In [ ]:
while len(train_data_partial) < len(train_data):
    if len(train_data_partial.loc[1]) < 49:
        pred_len = 1
    elif len(train_data_partial.loc[1]) >= 49 and len(train_data_partial.loc[1]) < 72:
        pred_len = 23
    elif len(train_data_partial.loc[1]) == 72:
        pred_len =  24

    predictor = TimeSeriesPredictor(
        prediction_length= pred_len,
        target="Temp",
        known_covariates_names=["Temp_Box"],
        eval_metric="RMSE",
        ignore_time_index = True
    ).fit(train_data_partial,
            #hyperparameters = {"DeepAR" : {"hidden_size": ag.space.Int(20, 100),}},
            #hyperparameter_tune_kwargs="auto",
            hyperparameters = {"DeepAR" : {}})

    known_covariates_per_item = []
    for item_id in train_data.item_ids:
        future_timestamps = train_data.loc[item_id][len(train_data_partial.loc[1]): len(train_data_partial.loc[1])+pred_len].index.get_level_values("timestamp")
        index = pd.MultiIndex.from_product([[item_id], future_timestamps], names=["item_id", "timestamp"])
        known_covariates_per_item.append(pd.DataFrame(train_data["Temp_Box"].values[len(train_data_partial.loc[1]): len(train_data_partial.loc[1])+pred_len], index=index, columns=["Temp_Box"]))
    known_covariates = TimeSeriesDataFrame(pd.concat(known_covariates_per_item))
    predict_data = predictor.predict(train_data_partial, known_covariates = known_covariates)
    new_train_data_per_item = []
    for item_id in train_data.item_ids:
        future_timestamps = train_data.loc[item_id][len(train_data_partial.loc[1]): len(train_data_partial.loc[1])+pred_len].index.get_level_values("timestamp")
        index = pd.MultiIndex.from_product([[item_id], future_timestamps], names=["item_id", "timestamp"])
        together = pd.concat([pd.DataFrame(train_data.loc[item_id]["Temp_Box"].values[len(train_data_partial.loc[1]): len(train_data_partial.loc[1])+pred_len], columns = ["Temp_Box"]), pd.DataFrame(predict_data.loc[item_id]['mean'].values, columns = ["Temp"])], axis = 1)
        together = pd.concat([train_data_partial[(item_id-1)*len(train_data_partial.loc[1]) : (item_id)*len(train_data_partial.loc[1])],together.set_index(index)], axis = 0)

        new_train_data_per_item.append(together)

    train_data_partial = TimeSeriesDataFrame(pd.concat(new_train_data_per_item))
train_data_partial